<a href="https://colab.research.google.com/github/Tclack88/DS-Unit-2-Kaggle-Challenge/blob/master/module3/assignment_kaggle_challenge_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science, Unit 2: Predictive Modeling

# Kaggle Challenge, Module 3


## Assignment
- [x] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [x] Continue to participate in our Kaggle challenge. 
- [x] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [x] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [x] Commit your notebook to your fork of the GitHub repo.

## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### More Categorical Encodings

**1.** The article **[Categorical Features and Encoding in Decision Trees](https://medium.com/data-design/visiting-categorical-features-and-encoding-in-decision-trees-53400fa65931)** mentions 4 encodings:

- **"Categorical Encoding":** This means using the raw categorical values as-is, not encoded. Scikit-learn doesn't support this, but some tree algorithm implementations do. For example, [Catboost](https://catboost.ai/), or R's [rpart](https://cran.r-project.org/web/packages/rpart/index.html) package.
- **Numeric Encoding:** Synonymous with Label Encoding, or "Ordinal" Encoding with random order. We can use [category_encoders.OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html).
- **One-Hot Encoding:** We can use [category_encoders.OneHotEncoder](http://contrib.scikit-learn.org/categorical-encoding/onehot.html).
- **Binary Encoding:** We can use [category_encoders.BinaryEncoder](http://contrib.scikit-learn.org/categorical-encoding/binary.html).


**2.** The short video 
**[Coursera — How to Win a Data Science Competition: Learn from Top Kagglers — Concept of mean encoding](https://www.coursera.org/lecture/competitive-data-science/concept-of-mean-encoding-b5Gxv)** introduces an interesting idea: use both X _and_ y to encode categoricals.

Category Encoders has multiple implementations of this general concept:

- [CatBoost Encoder](http://contrib.scikit-learn.org/categorical-encoding/catboost.html)
- [James-Stein Encoder](http://contrib.scikit-learn.org/categorical-encoding/jamesstein.html)
- [Leave One Out](http://contrib.scikit-learn.org/categorical-encoding/leaveoneout.html)
- [M-estimate](http://contrib.scikit-learn.org/categorical-encoding/mestimate.html)
- [Target Encoder](http://contrib.scikit-learn.org/categorical-encoding/targetencoder.html)
- [Weight of Evidence](http://contrib.scikit-learn.org/categorical-encoding/woe.html)

Category Encoder's mean encoding implementations work for regression problems or binary classification problems. 

For multi-class classification problems, you will need to temporarily reformulate it as binary classification. For example:

```python
encoder = ce.TargetEncoder(min_samples_leaf=..., smoothing=...) # Both parameters > 1 to avoid overfitting
X_train_encoded = encoder.fit_transform(X_train, y_train=='functional')
X_val_encoded = encoder.transform(X_train, y_val=='functional')
```

**3.** The **[dirty_cat](https://dirty-cat.github.io/stable/)** library has a Target Encoder implementation that works with multi-class classification.

```python
 dirty_cat.TargetEncoder(clf_type='multiclass-clf')
```
It also implements an interesting idea called ["Similarity Encoder" for dirty categories](https://www.slideshare.net/GaelVaroquaux/machine-learning-on-non-curated-data-154905090).

However, it seems like dirty_cat doesn't handle missing values or unknown categories as well as category_encoders does. And you may need to use it with one column at a time, instead of with your whole dataframe.

**4. [Embeddings](https://www.kaggle.com/learn/embeddings)** can work well with sparse / high cardinality categoricals.

_**I hope it’s not too frustrating or confusing that there’s not one “canonical” way to encode categorcals. It’s an active area of research and experimentation! Maybe you can make your own contributions!**_

### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [0]:
import os, sys
in_colab = 'google.colab' in sys.modules

# If you're in Colab...
if in_colab:
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge.git
    !git pull origin master
    
    # Install required python packages
    !pip install -r requirements.txt
    
    # Change into directory for module
    os.chdir('module3')

In [0]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv('../data/waterpumps/train_features.csv'), 
                 pd.read_csv('../data/waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv('../data/waterpumps/test_features.csv')
sample_submission = pd.read_csv('../data/waterpumps/sample_submission.csv')

### supporting definitions for cleaning and wrangling

In [0]:
import numpy as np
import matplotlib.pyplot as plt

In [0]:
# Supporting definitions for Wrangle
# this will lower the high cardinality columns, removing much of the noise
def convert_top_100(col_list,df,train):
  for col in col_list:
    global col_top_100
    col_top_100 = train[col].value_counts().index[0:100] # get top 100 of train only, then apply mapping to df in question if applicable
    df[col] = df[col].apply(map_to_100)
  return df
    
def map_to_100(feat):
  if feat in col_top_100:
    return feat
  else:
    return 'other'


  
  
# The main data wrangling function
def wrangle(df):
  
  # Imputer maintains True False, instead of 1,0. I prefer 1 column instead of 4
  bool_cols = ['public_meeting','permit']
  for col in bool_cols:
    df[col] = df[col].fillna(df[col].value_counts().index[0]).astype(int)
                 
  # convert some values to booleans, fill in missing values, replace suspicious values with nans
  high_cardinality_cols = ['wpt_name','subvillage','installer','funder','ward']
  df = convert_top_100(high_cardinality_cols,df,train) # keep the data pure to the "train" data set
  
  df.scheme_manamement = df.scheme_management.fillna('None') # a 'None' category exists, so might as well change nan to None
  
   # change this large count of near-zero outliers to 0, then convert to nan
  df.latitude = df.latitude.replace(-2e-8,0).value_counts()
  
  cols_with_0 = ['latitude','longitude', 'population','construction_year'] # population 1 also exists
  for col in cols_with_0:
    df[col] = df[col].replace(0,np.nan)
    
  df.population = df.population.replace(1,np.nan)

                 
  # remove columns that are "obviously" irrelevant, have too many categorical features, or are redundant
  drop = ['id','date_recorded','num_private','recorded_by','extraction_type',
          'extraction_type_group','water_quality','quantity','source','source_type',
          'waterpoint_type','scheme_name',
          'payment','management']
  
  df = df.drop(columns = drop)
  
  return df

### clean and wrangle data

In [62]:
train_id = train.id
test_id = test.id

y_train = train.status_group
train = train.drop(columns='status_group')
X_train = wrangle(train)
X_test = wrangle(test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access


## create model and parameters

In [0]:
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint,uniform

In [0]:
rf = make_pipeline(ce.OrdinalEncoder(),
                   SimpleImputer(),
                   RandomForestClassifier())

param_distributions = { 'simpleimputer__strategy':['mean','median','most_frequent'],
                       'randomforestclassifier__n_estimators' : randint(100,500),
                       'randomforestclassifier__max_depth' : [20,50,None],
                       'randomforestclassifier__min_weight_fraction_leaf' : uniform(.001,.1)
                      }

search = RandomizedSearchCV(rf,
                            param_distributions = param_distributions,
                            n_jobs = -1,
                            n_iter = 100,
                            scoring = 'accuracy',
                           verbose = 10,
                            cv = 5
                           )

In [66]:
search.fit(X_train,y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   17.4s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   57.3s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done  57 tasks      | elapsed:  7.1min
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:  8.9min
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed: 11.2min
[Parallel(n_jobs=-1)]: Done  94 tasks      | elapsed: 13.4min
[Parallel(n_jobs=-1)]: Done 109 tasks      | elapsed: 14.6min
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed: 16.2min
[Parallel(n_jobs=-1)]: Done 141 tasks      | elapsed: 19

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=Pipeline(memory=None,
                                      steps=[('ordinalencoder',
                                              OrdinalEncoder(cols=None,
                                                             drop_invariant=False,
                                                             handle_missing='value',
                                                             handle_unknown='value',
                                                             mapping=None,
                                                             return_df=True,
                                                             verbose=0)),
                                             ('simpleimputer',
                                              SimpleImputer(add_indicator=False,
                                                            copy=True,
                                                            f

In [69]:
search.best_params_

{'randomforestclassifier__max_depth': 50,
 'randomforestclassifier__min_weight_fraction_leaf': 0.0010931576312369065,
 'randomforestclassifier__n_estimators': 294,
 'simpleimputer__strategy': 'most_frequent'}

In [70]:
search.best_score_

0.7702188552188552

### make model and predictions, save submission

In [0]:
from sklearn.metrics import mean_absolute_error

In [0]:
pipeline = search.best_estimator_

y_pred = pipeline.predict(X_test)

In [0]:
submission = pd.concat([test_id,pd.Series(y_pred)],axis=1).rename(columns={0:'status_group'})
submission.to_csv("Trevor_waterpumps_sub_4.csv",index=False)